In [1]:
import pandas as pd 
import cv2                 
import numpy as np         
import os                  
from random import shuffle
from tqdm import tqdm  
import scipy
import skimage
from skimage.transform import resize


In [2]:
print(os.listdir("./data/"))

['test', 'train', 'val']


In [3]:

TRAIN_DIR = "./data/train/"
TEST_DIR =  "./data/test/"

In [4]:
def get_label(Dir):
    for nextdir in os.listdir(Dir):
        if not nextdir.startswith('.'):
            if nextdir in ['benign']:
                label = 0
            elif nextdir in ['malign']:
                label = 1
            else:
                label = 2
    return nextdir, label

In [5]:
def preprocessing_data(Dir):
    X = []
    y = []
    
    for nextdir in os.listdir(Dir):
        nextdir, label = get_label(Dir)
        temp = Dir + nextdir
        
        for image_filename in tqdm(os.listdir(temp)):
            path = os.path.join(temp + '/' , image_filename)
            img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
            if img is not None:
                img = skimage.transform.resize(img, (150, 150, 3))
                img = np.asarray(img)
                X.append(img)
                y.append(label)
            
    X = np.asarray(X)
    y = np.asarray(y)
    
    return X,y

In [6]:

def get_data(Dir):
    X = []
    y = []
    for nextDir in os.listdir(Dir):
        if not nextDir.startswith('.'):
            if nextDir in ['benign']:
                label = 0
            elif nextDir in ['malign']:
                label = 1
            else:
                label = 2
                
            temp = Dir + nextDir
                
            for file in tqdm(os.listdir(temp)):
                img = cv2.imread(temp + '/' + file)
                if img is not None:
                    img = skimage.transform.resize(img, (150, 150, 3))
                    #img_file = scipy.misc.imresize(arr=img_file, size=(150, 150, 3))
                    img = np.asarray(img)
                    X.append(img)
                    y.append(label)
                    
    X = np.asarray(X)
    y = np.asarray(y)
    return X,y

In [7]:
X_train, y_train = get_data(TRAIN_DIR)

100%|██████████| 62/62 [00:01<00:00, 53.93it/s]


In [8]:
X_test , y_test = get_data(TEST_DIR)

100%|██████████| 20/20 [00:00<00:00, 51.09it/s]


In [9]:
print(X_train.shape,'\n',X_test.shape)

(90, 150, 150, 3) 
 (30, 150, 150, 3)


In [10]:
print(y_train.shape, '\n',y_test.shape)

(90,) 
 (30,)


In [13]:
from keras.utils.np_utils import to_categorical
y_train = to_categorical(y_train,2)
y_test = to_categorical(y_test, 2)

ModuleNotFoundError: No module named 'keras'

In [14]:

Bimages = os.listdir(TRAIN_DIR + "benign")
Mimages = os.listdir(TRAIN_DIR + "malign")

FileNotFoundError: [WinError 3] Sistem belirtilen yolu bulamıyor: './data/train/benign'

In [ ]:
import matplotlib.pyplot as plt
def plotter(i):
    imagep1 = cv2.imread(TRAIN_DIR+"benign/"+Bimages[i])
    imagep1 = skimage.transform.resize(imagep1, (150, 150, 3) , mode = 'reflect')
    imagen1 = cv2.imread(TRAIN_DIR+"malign/"+Mimages[i])
    imagen1 = skimage.transform.resize(imagen1, (150, 150, 3))
    pair = np.concatenate((imagen1, imagep1), axis=1)
    print("(Left) - benign Vs (Right) - malign")
    print("-----------------------------------------------------------------------------------------------------------------------------------")
    plt.figure(figsize=(10,2))
    plt.imshow(pair)
    plt.show()
for i in range(1,3):
    plotter(i)

In [ ]:
from keras.callbacks import ReduceLROnPlateau , ModelCheckpoint , LearningRateScheduler
lr_reduce = ReduceLROnPlateau(monitor='val_acc', factor=0.1, min_delta=0.0001, patience=1, verbose=1)

In [ ]:
from keras.models import Sequential , Model
from keras.layers import Dense , Activation
from keras.layers import Dropout , GlobalAveragePooling2D
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD , RMSprop , Adadelta , Adam
from keras.layers import Conv2D , BatchNormalization
from keras.layers import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K 
#K.set_image_dim_ordering('th')
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

In [ ]:

X_train=X_train.reshape(7,3,150,150)
X_test=X_test.reshape(4,3,150,150)

In [ ]:
from keras.applications.inception_v3 import InceptionV3
# create the base pre-trained model
base_model = InceptionV3(weights=None, include_top=False , input_shape=(3, 150, 150))